In [25]:
import pandas as pd
import csv
from statistics import median
from projects import project_list
from batching_algs import *

In [26]:
r_file_name = 'ci_skipped_static_baseline_batching.csv'
result_file = open(r_file_name, 'w')
result_headers = ['project', 'algorithm', 'batch_size', 'builds_saved', 'median_delay', 'testall_size']
writer = csv.writer(result_file)
writer.writerow(result_headers)

69

In [27]:
batch_total = 0

In [28]:
def output_values(tr_results):
    l = []
    i = 0
    while i < len(tr_results):
        if tr_results[i] == 'passed':
            l.append(1)
        else:
            l.append(0)
        i += 1
    
    return l

In [29]:
def batching(project, data, algorithm, string):
    global batch_total
    
    p_name = project.split('/')[1]
    
    result_file = open('ci_skipped_static_baseline_batching.csv', 'a+')
    writer = csv.writer(result_file)
    
    results = output_values(data['tr_status'].tolist())
    length = len(results)
    
    if string == 'BATCHBISECT':
        batch_sizes = [2, 4, 8, 16]
    elif string == 'BATCHSTOP4':
        batch_sizes = [4, 8, 16]
    else:
        batch_sizes = [4, 8, 16]
    
    
       
    for b in batch_sizes:
        builds = 0
        delays = []
        
        i = 0
        while i < length:
            if length - i > b:
                batch = results[i:i+b]
                delays.extend([b-x-1 for x in range(b)])
            else:
                batch = results[i:]
                delays.extend([b-x-1 for x in range(length-i)])
            batch_total = algorithm(batch)
            builds += batch_total
            #print(b, batch, batch_total, builds)
            
            i += b
        
        builds_saved = 100-(100*builds/length)
        median_delay = median(delays)
        
        writer.writerow([p_name, string, b, builds_saved, median_delay, length])

In [30]:
discard_list = []
for p in project_list:
    p_name = p.split('/')[1]
    try:
        data = pd.read_csv('ci_skip_data/extracted_project_travis/'+p_name)
    except:
        continue
    
    branch_type = data['git_branch'].tolist()
    if 'master' in branch_type:
        data = data[ data['git_branch'] == 'master']
    else:
        data = data[ data['git_branch'] == 'trunk']
    
    data = data[ data['verdict'] == 0]
    data = data.iloc[100:]
    if len(data) == 0:
        print('Could not continue for {}'.format(p))
        discard_list.append(p)
        continue
    
    batching(p, data, batchbisect, 'BATCHBISECT')
    batching(p, data, batchstop4, 'BATCHSTOP4')
    batching(p, data, batchdivide4, 'BATCHDIVIDE4')    

Could not continue for BuildCraft-BuildCraft/BuildCraft-BuildCraft.csv
